In [5]:
# Start with exploring Deep Neural Networks 

In [6]:
import tensorflow as tf 
import numpy as np
from sklearn.preprocessing import StandardScaler
# Construction Phase:

#specify inputs outputs number of neurons on each layer

n_inputs = 28*28
n_hidden1 =300
n_hidden2 = 100
n_outputs =10 

#define our X and y placeholders size set to none for now b/c don't know training data size

X = tf.placeholder(tf.float32, shape = (None , n_inputs) , name = "X")
y = tf.placeholder(tf.int64, shape = (None),name = "y")

"""X will act as the input layer, when executed it will be replaced with one
training batch at a time"""

#Create a function that will make one neuron layer at a time 

def neuron_layer(X, n_neurons, name, activation =None):
    with tf.name_scope(name): #contains compuation nodes for neuron layer
        n_inputs = int(X.get_shape()[1]) #find num rows in matrix (2nd dimension)
        
        #Create the weights matrix
        
        """Weights matrix is a 2D tensor containing all the weights between
        each neuron and each input. Hence shape is (n_inputs, n_neurons). Initi
        -alized randomly using a normal distribution """
        stddev = 2 / np.sqrt(n_inputs) 
        init = tf.truncated_normal((n_inputs, n_neurons), stddev =stddev)
        W = tf.Variable(init, name = "kernel")
        
        #Create biases
        b = tf.Variable(tf.zeros([n_neurons]), name = "bias")
        #compute weighted sums + biases for rach neuro in the layer
        Z = tf.matmul(X,W)+b
        
        #Execute if a specific output activation is provided 
        if activation is not None:
            return activation(Z)
        else:
            return Z
        

#Create DNN!

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name = "Hidden1", activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = "Hidden2", activation = tf.nn.relu)
    
    logits = neuron_layer(hidden2, n_outputs, name = "outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y , logits=logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
#last step: specify how to evaluate the model 

#use accuracy as performance measure

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

#create node to initialize all variables and create a saver

init = tf.global_variables_initializer()
saver = tf.train.Saver()



